### Loading the Boston housing dataset

In [54]:
from keras.datasets import boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

time: 5.68 ms


In [55]:
print(test_data.shape)

(102, 13)
time: 729 µs


### Preparing the data

Normalizing the data

In [56]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

time: 2.71 ms


### Building your network

In [58]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras.losses import mse
from keras.metrics import mae


def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))

  model.compile(optimizer=RMSprop(learning_rate=0.001),
                loss = mse,
                metrics = [mae])
  
  return model

time: 9.23 ms


### Validating your approach using K-fold validation

In [ ]:
# sample code for k-fold validation
'''
import numpy as np

train_data = [[1,2,3,4,5,6,7,8,9,10,11,12,13],
              [2,5,6,8,2,4,5,7,1,5,7,8,43, 2],
              [3,8,4,3,3,7,6,5,4,3,2,1,7, 6],
              [4,2,3,4,5,6,7,8,9,10,11,12,13],
              [5,5,6,8,2,4,5,7,1,5,7,8,43, 2],
              [6,8,4,3,3,7,6,5,4,3,2,1,7, 6],
              [7,2,3,4,5,6,7,8,9,10,11,12,13],
              [8,5,6,8,2,4,5,7,1,5,7,8,43, 2],
              [9,8,4,3,3,7,6,5,4,3,2,1,7, 6],
              [10,2,3,4,5,6,7,8,9,10,11,12,13],
              [11,5,6,8,2,4,5,7,1,5,7,8,43, 2],
              [12,8,4,3,3,7,6,5,4,3,2,1,7, 6]]

train_labels = [101,102,103,104,105,106,107,108,109,110,111,112]

k = 4
num_val_samples = len(train_data) // k
# print("num_val_samples:", num_val_samples)
num_epochs = 100
all_scores = []

for i in range(k):
  print("processing fold #", i)
  print(f"val_data from {i*num_val_samples} to {(i+1)*num_val_samples} ")
  val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets = train_labels[i*num_val_samples: (i+1)*num_val_samples]
  print("val_data: ",val_data)
  print("val_targets: ",val_targets)
  print()
  print(f"partial_train_data from {train_data[:i * num_val_samples]} to {train_data[(i + 1) * num_val_samples:]}")
  partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_labels[:i * num_val_samples], train_labels[(i + 1) * num_val_samples:]], axis=0)
  print("partial_train_data: ", partial_train_data)
  print("partial_train_targets: ", partial_train_targets)
  print("####################################################")
  print()
  '''

In [59]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
  val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets = train_labels[i*num_val_samples: (i+1)*num_val_samples]

  partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_labels[:i * num_val_samples], train_labels[(i + 1) * num_val_samples:]], axis=0)

  model = build_model()
  model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

time: 4min 4s


In [65]:
print("Mean Absolute Error: ",np.array(all_scores).mean())

Mean Absolute Error:  2.4825117588043213
time: 1.19 ms
